In [1]:
import re

def clean_description(description):    
    description = re.sub(r'http\S+', '', description)
    description = re.sub(r'\S*@\S*\s?', '', description)
    description = re.sub(r'#', '', description)
    description = re.sub(r'(?<![^ .,?!;])podcast(?![^ .:,?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])podcasts(?![^ .,:?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])host(?![^ .,?!:;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])hosts(?![^ .,?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])episode(?![^ .,:?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])episodes(?![^ .:,?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])sponsor(?![^ .,:?!;\r\n])', '', description)
    description = re.sub(r'(?<![^ .,?!;])sponsored(?![^ .:,?!;\r\n])', '', description)
    return description.lower()


In [2]:
import pandas

# get only the necessary columns from the data and convert to a dataframe
podcasts = pandas.read_csv('metadata.tsv', sep='\t', usecols = ['show_uri', 'show_name', 'language', 'show_description', 'rss_link', 'episode_uri', 'episode_name',	'episode_description',	'duration'], nrows=12000)

# only include podcasts in english language
eng_abr = set(("['en']", "['en-au']", "['en-bz'}", "['en-ca']", "['en-ie']", "['en-jm']", "['en-nz']", "['en-za']", "['en-tt']", "['en-gb']", "['en-us']"))

# ensure all values in language column are lowercase
podcasts['language'] = podcasts['language'].str.lower()

# create dataframe of english podcasts
eng_podcasts = podcasts[podcasts['language'].isin(eng_abr)]
eng_podcasts = eng_podcasts.drop(['language'], axis=1)
eng_podcasts = eng_podcasts[eng_podcasts['episode_description'].apply(lambda x: isinstance(x, str))]
eng_podcasts = eng_podcasts[eng_podcasts['show_description'].apply(lambda x: isinstance(x, str))]
eng_podcasts['image_url'] = ''

In [4]:
# categorize the podcasts and create category dataframe
# open all xml files in show-rss
import os
import xml.etree.ElementTree as ET
import requests
def parseRSS(rss_link):
   success = False
   ns = {'itunes': 'http://www.itunes.com/dtds/podcast-1.0.dtd'}
   try:
      resp = requests.get(rss_link)
   except:
      resp = None
   if resp != None:
      with open('podcast.xml', 'wb') as f:
         f.write(resp.content)
      categories = set()
      image_url = None
      try:
         # create element tree object
         tree = ET.parse('podcast.xml')
         # get root element
         root = tree.getroot()
         for child in root.iter():
            for cat in child.findall('itunes:category', ns):
               category = cat.get('text')
               categories.add(category)
            image_tag = child.findall('itunes:image', ns)
            if image_tag:
               image_url = image_tag[0].get('href')
         if len(categories) > 0 and image_url and ('http' in image_url) and ('.jpg' in image_url or '.png' in image_url):
            success = True
      except:
         pass
      finally:
         if os.path.exists("podcast.xml"):
            os.remove("podcast.xml")
         if success:
            return (categories, image_url)
   return (None, None)


# create a list of categories and the associated podcast
cat_list = list()
count = 0
all_categories = set()
indices_to_remove = []
# iterate through all columns in dataframe
for ind in eng_podcasts.index:
   if count < 10000:
      uri = eng_podcasts['episode_uri'][ind]
      (categories, image_url) = parseRSS(eng_podcasts['rss_link'][ind])
      if categories != None and image_url != None:
         count += 1
         for cat in categories:
            all_categories.add(cat)
            cat_list.append({'episode_uri': uri, 'category': cat})
         eng_podcasts.loc[ind, 'image_url'] = image_url
      else:
         indices_to_remove.append(ind)
   else:
      indices_to_remove.append(ind)

# remove the rows in dataframe that were not succesful
# eng_podcasts = eng_podcasts.drop(eng_podcasts.index[indices_to_remove])
            
# convert list of categories and podcast to dataframe
cat_df = pandas.DataFrame.from_records(cat_list)
      

In [4]:
import dill
dill.load_session('notebook_env.db')

In [18]:
# create subcategories dataframe
from rake_nltk import Rake
import nltk

subcat_list = list()
for index in eng_podcasts.index:
    episode_uri = eng_podcasts['episode_uri'][index]
    episode_desc = clean_description(eng_podcasts['episode_description'][index])
    show_desc = clean_description(eng_podcasts['show_description'][index])
    episode_words = set(episode_desc.split(' '))
    show_words = set(show_desc.split(' '))
    descriptions = episode_desc + episode_desc + episode_desc + show_desc
    r = Rake()
    r.extract_keywords_from_text(descriptions)
    key_phrases = r.get_ranked_phrases_with_scores()
    count_subcategories = 0
    cur_score = 0
    cur_subcategories = set()
    while key_phrases:
        cur_phrase = key_phrases.pop(0)
        if count_subcategories >= 10 or (count_subcategories >= 5 and cur_score != cur_phrase[0]):
            break
        cur_score = cur_phrase[0]
        if cur_phrase[1] in cur_subcategories:
            continue
        cur_subcategories.add(cur_phrase[1].rstrip())
        is_power_subcategory = False
        if cur_phrase[1].lower() in show_desc and cur_phrase[1] in episode_desc:
            is_power_subcategory = True
        new_item = {'episode_uri': episode_uri, 'subcategory': cur_phrase[1].rstrip(), 'is_power': is_power_subcategory}
        subcat_list.append(new_item)
        count_subcategories += 1

subcat_df = pandas.DataFrame.from_records(subcat_list)

In [9]:
descriptions = 'Hello my name is Meredith. I am very curious why my stuff will not work. Do you have any ideas? Soccer ahs long been my favorite sport. I play center midfield on my soccer team.'
r = Rake()
r.extract_keywords_from_text(descriptions)
key_phrases = r.get_ranked_phrases_with_scores()
print(key_phrases)

[(9.0, 'play center midfield'), (8.5, 'soccer ahs long'), (4.5, 'soccer team'), (4.0, 'favorite sport'), (1.0, 'work'), (1.0, 'stuff'), (1.0, 'name'), (1.0, 'meredith'), (1.0, 'ideas'), (1.0, 'hello'), (1.0, 'curious')]


In [14]:
import dill
dill.dump_session('notebook_env.db')

In [22]:
import sqlalchemy
import pandas as pd

hostname="localhost"
dbname="mheller5"
uname="mheller5"
pwd="audioodyssey"

podcast_df = eng_podcasts.drop(['rss_link', 'episode_description', 'show_description', 'image_url'], axis=1)
# make episode description table
desc_df = eng_podcasts[['episode_uri', 'episode_description']]
image_df = eng_podcasts[['episode_uri', 'image_url']]


# Create SQLAlchemy engine to connect to MySQL Database
engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

# Convert dataframe to sql table                                   
podcast_df.to_sql('podcasts', engine, index=False, dtype={
	# 'episode_uri': sqlalchemy.types.NVARCHAR(length=40),
	# 'show_uri': sqlalchemy.types.NVARCHAR(length=40),
	# 'show_name': sqlalchemy.types.NVARCHAR(length=255),
	# 'episode_name': sqlalchemy.types.NVARCHAR(length=255),
	# 'duration': sqlalchemy.types.FLOAT, # TODO: need to figure out this type
	# 'rss_link': sqlalchemy.types.NVARCHAR(length=70)
})
desc_df.to_sql('descriptions', engine, dtype={'episode_uri': sqlalchemy.types.NVARCHAR(length=40)}, index=False)
image_df.to_sql('image_urls', engine, dtype={'episode_uri': sqlalchemy.types.NVARCHAR(length=40)}, index=False)
cat_df.to_sql('categories', engine, index=False, dtype={
	'episode_uri': sqlalchemy.types.NVARCHAR(length=40),
	'category': sqlalchemy.types.NVARCHAR(length=40)
})
# subcat_df.to_sql('subcategories', engine, index=False)


16376

In [19]:
import sqlalchemy
import pandas as pd

hostname="localhost"
dbname="mheller5"
uname="mheller5"
pwd="audioodyssey"

# Create SQLAlchemy engine to connect to MySQL Database
engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

# make sure all subcats are strings
subcat_df = subcat_df[subcat_df['subcategory'].apply(lambda x: isinstance(x, str))]

subcat_df.to_sql('subcategories', engine, index=False, dtype={
	'episode_uri': sqlalchemy.types.NVARCHAR(length=40),
	'is_power': sqlalchemy.types.BOOLEAN
})

63830